In [9]:
# medical_chatbot.py

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
import streamlit as st

# 1. Load PDF
loader = PyPDFLoader('C:/Users/DELL/Documents/Projects/The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf')
documents = loader.load()

# 2. Split text into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = splitter.split_documents(documents)

# 3. Use HuggingFace embeddings
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 4. Create FAISS vectorstore
vectorstore = FAISS.from_documents(docs, embedding)

# 5. Setup conversation chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),  # You can replace with local LLM if needed
    retriever=vectorstore.as_retriever(),
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)
)

# 6. Streamlit UI
st.set_page_config(page_title="Medical Chatbot", layout="wide")
st.title("🩺 Gale Encyclopedia Medical Chatbot")

if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

user_query = st.text_input("Enter your health-related question:")

if user_query:
    response = qa_chain.run(user_query)
    st.session_state.chat_history.append((user_query, response))

if st.session_state.chat_history:
    for question, answer in reversed(st.session_state.chat_history):
        st.markdown(f"**🧑 You:** {question}")
        st.markdown(f"**🤖 Bot:** {answer}")


C:\Users\DELL\AppData\Local\Temp\ipykernel_5308\506241491.py:21: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).